In [1]:
import requests as req
import pandas as pd

In [42]:
endpoint_vacancies = 'https://api.hh.ru/vacancies'
api_headers = {'User-Agent': 'Mozilla/5.0',
               'Authorization': 'Bearer YOUR_API_KEY'}
api_params = {'text':'python middle',
              'per_page':100,
              'page':9}

In [14]:
vacancyDetailInfo =[]

In [43]:
results = req.get(endpoint_vacancies, params = api_params)

In [44]:
results.text

'{"items":[{"id":"81754071","premium":false,"name":"Senior / Middle+ QA Automation (python)","department":null,"has_test":false,"response_letter_required":false,"area":{"id":"1","name":"Москва","url":"https://api.hh.ru/areas/1"},"salary":null,"type":{"id":"open","name":"Открытая"},"address":{"city":"Москва","street":"Рочдельская улица","building":"15с27","lat":55.754836,"lng":37.560565,"description":null,"raw":"Москва, Рочдельская улица, 15с27","metro":{"station_name":"Краснопресненская","line_name":"Кольцевая","station_id":"5.58","line_id":"5","lat":55.760378,"lng":37.577114},"metro_stations":[{"station_name":"Краснопресненская","line_name":"Кольцевая","station_id":"5.58","line_id":"5","lat":55.760378,"lng":37.577114},{"station_name":"Улица 1905 года","line_name":"Таганско-Краснопресненская","station_id":"7.146","line_id":"7","lat":55.763944,"lng":37.562271}],"id":"11116713"},"response_url":null,"sort_point_distance":null,"published_at":"2023-06-21T13:50:35+0300","created_at":"2023-06

In [26]:
for result in results['items']:
    vacancyRow = []
    listOfRequirements = []
    vacancyName = result['name']
    companyName = result['employer']['name']
    vacancyid = result['id']
    vacancyDetail = req.get(endpoint_vacancies+'/'+vacancyid).json()
    vacancyDescription = vacancyDetail['description']
    for skill in vacancyDetail['key_skills']:
        listOfRequirements.append(skill['name'])
    vacancyRow.append(companyName)
    vacancyRow.append(vacancyName)
    vacancyRow.append(vacancyDescription)
    vacancyRow.append(listOfRequirements)
    vacancyDetailInfo.append(vacancyRow)


In [20]:
vacancyDetailInfo

[['Интернет вещей',
  'Junior/Middle Python разработчик',
  '<p>Мы ищем опытного Python разработчика, который присоединится к нашей команде разработки. Ваша работа будет состоять из разработки и поддержки нашего продуктов на Python. Вы будете работать в тесном сотрудничестве с командой разработки, чтобы создавать высококачественный код и улучшать наш продукт.</p> <p> </p> <p><strong>Основные обязанности:</strong></p> <ul> <li>Разработка и поддержка нашего продукта на Python</li> <li>Создание высококачественного и надежного кода</li> <li>Участие в разработке архитектуры приложения</li> <li>Тестирование и отладка приложения</li> <li>Интеграция с другими сервисами и API</li> </ul> <p> </p> <p><strong>Требования:</strong></p> <ul> <li>Опыт работы в разработке на Python от 2 лет</li> <li>Знание Flask/Django и опыт работы с веб-фреймворками</li> <li>Опыт работы с базами данных (PostgreSQL, MongoDB)</li> <li>Опыт работы с Git и системами контроля версий</li> <li>Опыт работы с Linux и bash</li

In [27]:
columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=columns,index=None)

In [28]:
df

,company_name,position,job_description,key_skills
0,Интернет вещей,Junior/Middle Python разработчик,"<p>Мы ищем опытного Python разработчика, котор...","[Python, PostgreSQL, API, REST API, Django, Dj..."
1,Дион Софт,QA Engineer/Тестировщик (Middle),"<p><strong>ООО «ДИОН СОФТ» компания, занимающа...","[SQL, PostgreSQL, Регрессионное тестирование, ..."
2,ГРЕЧКА,Backend - разработчик на Python,<p>Мы - Компания <strong>Гречка.Digital</stron...,"[Python, Django Framework, Git, PostgreSQL, My..."
3,НИИАС,Junior/Middle Python-developer,<p><strong>АО «НИИАС» (дочернее общество ОАО «...,[]
4,Notamedia,Manual QA инженер (middle),"<p>Notamedia — аккредитованный ИТ-холдинг, оди...","[Тестирование, Postman, Swagger, Charles, Atla..."
...,...,...,...,...
195,Виго Лаб,Senior/Middle Java Developer,<p><strong>Какую проблему решаем:</strong></p>...,[]
196,СоларЛаб,DevOps инженер middle,"<p>Мы ищем DevOps специалиста, который готов в...","[CI, CI/CD, Docker, Bash, Jenkins, Gitlab CI, ..."
197,СБЕР,Аналитик данных,<p><strong><em>Мы - команда Управления противо...,[]
198,СБЕР,Localization & Mapping Engineer в центр робото...,<p>Мы команда робототехники Центра Робототехни...,[]


In [ ]:
results['key_skills'].append('s')

In [ ]:
for skill in results['key_skills']:
    skill['name']